In [2]:
import pandas as pd
import numpy as np

In [3]:
#De Json a dataframe
ruta = "datos_crudos/datos_santiago_crudos_2024.xml"

# Leer el archivo XML
try:
    df = pd.read_xml(ruta)
    print("Archivo XML cargado con éxito.")
    print(df.head())
    
except Exception as e:
    print(f"Error al leer el archivo XML: {e}")
    
#Eliminar columnas
df = pd.read_xml(ruta, xpath=".//registro", parser="etree")


df.drop(['snow', 'pres', 'tsun', 'wpgt'], axis='columns', inplace=True)


Archivo XML cargado con éxito.
                  time  tavg  tmin  tmax  prcp  snow  wdir  wspd  wpgt  \
0  2024-01-01 00:00:00  26.1  22.7  31.0   0.1   NaN   NaN  10.1   NaN   
1  2024-01-02 00:00:00  26.1  22.4  31.6   0.0   NaN   NaN  10.8   NaN   
2  2024-01-03 00:00:00  26.4  21.6  32.6   0.0   NaN   NaN   9.3   NaN   
3  2024-01-04 00:00:00  26.2  21.7  32.4   0.2   NaN   NaN   8.6   NaN   
4  2024-01-05 00:00:00  26.2  20.8  32.6   0.3   NaN   NaN   8.9   NaN   

     pres  tsun  
0  1011.9   NaN  
1  1011.4   NaN  
2  1011.0   NaN  
3  1011.8   NaN  
4  1011.0   NaN  


In [4]:
print(df.dtypes)

time     object
tavg    float64
tmin    float64
tmax    float64
prcp    float64
wdir    float64
wspd    float64
dtype: object


In [5]:
#Limpieza de datos
df_santiago = df.drop(columns=["Día","Año", "Mes"], errors='ignore')
df_santiago.replace(["", "-", "NA", "N/A", "n/a"], np.nan, inplace=True)
# Convertir la columna Date a datetime y dejar solo año-mes-día
df_santiago["time"] = pd.to_datetime(df_santiago["time"], errors="coerce").dt.strftime("%Y-%m-%d")


In [6]:
# Porcentaje de valores faltantes
faltantes = df_santiago.isna().sum()
print("Valores faltantes por columna:")
print(faltantes[faltantes > 0])
porcentaje_faltantes = (df_santiago.isna().sum() / len(df_santiago)) * 100
print("\nPorcentaje de valores faltantes:")
print(porcentaje_faltantes[porcentaje_faltantes > 0].round(2))

Valores faltantes por columna:
tavg     13
tmin     13
tmax     13
prcp     13
wdir    366
wspd     13
dtype: int64

Porcentaje de valores faltantes:
tavg      3.55
tmin      3.55
tmax      3.55
prcp      3.55
wdir    100.00
wspd      3.55
dtype: float64


In [7]:
#Limpieza de las columnas
for col in df_santiago.columns:
    if col != 'time':  # Evitar convertir la columna de fechas
        df_santiago[col] = pd.to_numeric(df_santiago[col], errors='coerce')

# Imputar temperaturas con la media
for col in ["tmax", "tmin", "tavg"]:
    df_santiago[col] = df_santiago[col].fillna(df_santiago[col].mean())

# Imputar precipitaciones con 0 = no llovió)
for col in ["prcp"]:
    df_santiago[col] = df_santiago[col].fillna(0)

# Imputar viento con la media 
for col in ["wspd"]:
    df_santiago[col] = df_santiago[col].fillna(df_santiago[col].mean())

# Imputar texto faltante con la moda (valor más frecuente)
for col in ["wdir"]:
    if df_santiago[col].isna().sum() > 0:
        moda = df_santiago[col].mode()
        if not moda.empty:
            df_santiago[col] = df_santiago[col].fillna(moda[0])

columnas_numericas = df_santiago.select_dtypes(include=["number"]).columns
df_santiago[columnas_numericas] = df_santiago[columnas_numericas].round(1)

In [8]:
#Cambiar el nombre de las columnas y agregar dos adicionales
columnas_estandar= {
    'time': 'Date',
    'tmax': 'Tmax',
    'tmin': 'Tmin',
    'tavg': 'Tavg',
    'prcp': 'Prcp',
    'wspd': 'Wspd',
    'wdir': 'Wdir'
}

# Renombrar columnas y eliminar las demás
df_santiago= df_santiago.rename(columns=columnas_estandar)  
df_santiago = df_santiago[list(columnas_estandar.values())] 
df_santiago["Estacion"] = "Santiago"
df_santiago["Provincia"] = "Veraguas"

In [9]:
print(df_santiago)

           Date  Tmax  Tmin  Tavg  Prcp  Wspd  Wdir  Estacion Provincia
0    2024-01-01  31.0  22.7  26.1   0.1  10.1   NaN  Santiago  Veraguas
1    2024-01-02  31.6  22.4  26.1   0.0  10.8   NaN  Santiago  Veraguas
2    2024-01-03  32.6  21.6  26.4   0.0   9.3   NaN  Santiago  Veraguas
3    2024-01-04  32.4  21.7  26.2   0.2   8.6   NaN  Santiago  Veraguas
4    2024-01-05  32.6  20.8  26.2   0.3   8.9   NaN  Santiago  Veraguas
..          ...   ...   ...   ...   ...   ...   ...       ...       ...
361  2024-12-27  27.2  22.5  24.4  12.3   7.2   NaN  Santiago  Veraguas
362  2024-12-28  28.6  22.7  24.4   3.8   7.4   NaN  Santiago  Veraguas
363  2024-12-29  27.8  22.3  24.2   5.2   9.6   NaN  Santiago  Veraguas
364  2024-12-30  29.3  21.4  24.3   9.1   6.7   NaN  Santiago  Veraguas
365  2024-12-31  28.6  21.9  24.0  17.1   5.1   NaN  Santiago  Veraguas

[366 rows x 9 columns]


In [ ]:
#Pasar a CSV
df_santiago.to_csv("datos_limpios/datos_limpios_santiago_2024.csv", index=False)